In [128]:
%cd F:\FDM Group Project

F:\FDM Group Project


In [129]:
!pip install missingpy
!pip install miceforest

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import miceforest as mf
import random
import sklearn.neighbors._base 
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from sklearn.impute import KNNImputer

In [131]:
data = pd.read_csv(r'Data\rental_price_data.csv')

In [132]:
data.head()

,accommodates,amenities,availability_30,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,cancellation_policy,guests_included,has_availability,host_is_superhost,host_listings_count,instant_bookable,latitude(North),longitude(East),maximum_nights,number_of_reviews,property_type,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,price
0,6,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",0,2.5,Real Bed,1.0,3.0,1,moderate,1,t,f,1.0,f,22.542900,114.059600,10,1,House,10.0,10.0,10.0,100.0,10.0,Private room,$300.00
1,2,"{""Air Conditioning"",Heating,""Family/Kid Friend...",29,0.0,Futon,1.0,1.0,1,moderate,1,t,f,1.0,f,22.539490,114.043225,7,0,Camper/RV,NaN,NaN,NaN,NaN,NaN,Private room,$99.00
2,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,1.5,Real Bed,1.0,1.0,2,flexible,1,t,f,2.0,f,22.508573,114.079426,1125,0,House,NaN,NaN,NaN,NaN,NaN,Private room,$100.00
3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,1.5,Real Bed,1.0,1.0,2,flexible,1,t,f,2.0,f,22.508697,114.079035,1125,0,House,NaN,NaN,NaN,NaN,NaN,Private room,$100.00
4,6,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",27,2.0,Real Bed,3.0,3.0,2,strict,1,t,f,301.0,f,22.509502,114.055590,1125,0,House,NaN,NaN,NaN,NaN,NaN,Entire home/apt,$599.00


In [133]:
#preprocess the amentities column.
#extracting the values as features.
amenities_key = set()
def extract_feature(x):
    s1 = x.replace('{', '').replace('}', '').replace('"', '')
    s1 = s1.split(',')
    for s in s1:
        amentities_key.add(s)
        #print(s)

data['amenities'].apply(extract_feature)

0       None
1       None
2       None
3       None
4       None
        ... 
5829    None
5830    None
5831    None
5832    None
5833    None
Name: amenities, Length: 5834, dtype: object

In [134]:
amenities_key

set()

In [135]:
amenities_key.remove('')

KeyError: ''

In [136]:
amenities_key

set()

In [137]:
#we store each value as a variable in the dataset.
#and then storing 1 for the variables if the amintities column is having the value. otherwise 0.
for amentities in amentities_key:
    data['amenities__'+amentities.replace(' ','_')] = data['amenities'].str.contains(amentities).astype(int)

C:\Users\thari\AppData\Local\Temp\ipykernel_18276\3560883285.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['amenities__'+amentities.replace(' ','_')] = data['amenities'].str.contains(amentities).astype(int)
C:\Users\thari\AppData\Local\Temp\ipykernel_18276\3560883285.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['amenities__'+amentities.replace(' ','_')] = data['amenities'].str.contains(amentities).astype(int)
C:\Users\thari\AppData\Local\Temp\ipykernel_18276\3560883285.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['amenities__'+amentities.replace(' ','_')] = data['amenities'].str.contains(amentities).astype(int)


In [138]:
#now all together we have 67 variables.
data.head()

,accommodates,amenities,availability_30,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,cancellation_policy,guests_included,has_availability,host_is_superhost,host_listings_count,instant_bookable,latitude(North),longitude(East),maximum_nights,number_of_reviews,property_type,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,price,amenities__Dryer,amenities__Other_pet(s),amenities__Carbon_Monoxide_Detector,amenities__,amenities__Iron,amenities__Elevator_in_Building,amenities__Cable_TV,amenities__Heating,amenities__Shampoo,amenities__Wireless_Internet,amenities__Dog(s),amenities__Doorman,amenities__24-Hour_Check-in,amenities__TV,amenities__First_Aid_Kit,amenities__Pets_live_on_this_property,amenities__Essentials,amenities__Laptop_Friendly_Workspace,amenities__Hair_Dryer,amenities__Gym,amenities__Safety_Card,amenities__Pool,amenities__Internet,amenities__Kitchen,amenities__Pets_Allowed,amenities__Buzzer/Wireless_Intercom,amenities__Breakfast,amenities__Washer_/_Dryer,amenities__Suitable_for_Events,amenities__Free_Parking_on_Premises,amenities__Lock_on_Bedroom_Door,amenities__Washer,amenities__Hot_Tub,amenities__Cat(s),amenities__Air_Conditioning,amenities__Indoor_Fireplace,amenities__Smoke_Detector,amenities__Wheelchair_Accessible,amenities__Family/Kid_Friendly,amenities__Fire_Extinguisher,amenities__Hangers,amenities__Smoking_Allowed
0,6,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",0,2.5,Real Bed,1.0,3.0,1,moderate,1,t,f,1.0,f,22.542900,114.059600,10,1,House,10.0,10.0,10.0,100.0,10.0,Private room,$300.00,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,1,0,0
1,2,"{""Air Conditioning"",Heating,""Family/Kid Friend...",29,0.0,Futon,1.0,1.0,1,moderate,1,t,f,1.0,f,22.539490,114.043225,7,0,Camper/RV,NaN,NaN,NaN,NaN,NaN,Private room,$99.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0
2,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,1.5,Real Bed,1.0,1.0,2,flexible,1,t,f,2.0,f,22.508573,114.079426,1125,0,House,NaN,NaN,NaN,NaN,NaN,Private room,$100.00,1,0,0,1,0,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,0,0
3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,1.5,Real Bed,1.0,1.0,2,flexible,1,t,f,2.0,f,22.508697,114.079035,1125,0,House,NaN,NaN,NaN,NaN,NaN,Private room,$100.00,1,0,0,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,0,0
4,6,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",27,2.0,Real Bed,3.0,3.0,2,strict,1,t,f,301.0,f,22.509502,114.055590,1125,0,House,NaN,NaN,NaN,NaN,NaN,Entire home/apt,$599.00,1,0,0,1,0,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0


In [139]:
#no need of amentities column now.
data.drop(['amenities'], axis=1, inplace=True)

In [142]:
#replace $ sign
data['price'] = data['price'].str.replace('$','')
#replace , sign
data['price'] = data['price'].str.replace(',','')

C:\Users\thari\AppData\Local\Temp\ipykernel_18276\3560235492.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['price'] = data['price'].str.replace('$','')


In [143]:
data.head()

,accommodates,availability_30,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,cancellation_policy,guests_included,has_availability,host_is_superhost,host_listings_count,instant_bookable,latitude(North),longitude(East),maximum_nights,number_of_reviews,property_type,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,price,amenities__Dryer,amenities__Other_pet(s),amenities__Carbon_Monoxide_Detector,amenities__,amenities__Iron,amenities__Elevator_in_Building,amenities__Cable_TV,amenities__Heating,amenities__Shampoo,amenities__Wireless_Internet,amenities__Dog(s),amenities__Doorman,amenities__24-Hour_Check-in,amenities__TV,amenities__First_Aid_Kit,amenities__Pets_live_on_this_property,amenities__Essentials,amenities__Laptop_Friendly_Workspace,amenities__Hair_Dryer,amenities__Gym,amenities__Safety_Card,amenities__Pool,amenities__Internet,amenities__Kitchen,amenities__Pets_Allowed,amenities__Buzzer/Wireless_Intercom,amenities__Breakfast,amenities__Washer_/_Dryer,amenities__Suitable_for_Events,amenities__Free_Parking_on_Premises,amenities__Lock_on_Bedroom_Door,amenities__Washer,amenities__Hot_Tub,amenities__Cat(s),amenities__Air_Conditioning,amenities__Indoor_Fireplace,amenities__Smoke_Detector,amenities__Wheelchair_Accessible,amenities__Family/Kid_Friendly,amenities__Fire_Extinguisher,amenities__Hangers,amenities__Smoking_Allowed
0,6,0,2.5,Real Bed,1.0,3.0,1,moderate,1,t,f,1.0,f,22.542900,114.059600,10,1,House,10.0,10.0,10.0,100.0,10.0,Private room,300.00,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,1,0,0
1,2,29,0.0,Futon,1.0,1.0,1,moderate,1,t,f,1.0,f,22.539490,114.043225,7,0,Camper/RV,NaN,NaN,NaN,NaN,NaN,Private room,99.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0
2,2,30,1.5,Real Bed,1.0,1.0,2,flexible,1,t,f,2.0,f,22.508573,114.079426,1125,0,House,NaN,NaN,NaN,NaN,NaN,Private room,100.00,1,0,0,1,0,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,0,0
3,2,30,1.5,Real Bed,1.0,1.0,2,flexible,1,t,f,2.0,f,22.508697,114.079035,1125,0,House,NaN,NaN,NaN,NaN,NaN,Private room,100.00,1,0,0,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,0,0
4,6,27,2.0,Real Bed,3.0,3.0,2,strict,1,t,f,301.0,f,22.509502,114.055590,1125,0,House,NaN,NaN,NaN,NaN,NaN,Entire home/apt,599.00,1,0,0,1,0,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0


In [144]:
data.dtypes

accommodates                          int64
availability_30                       int64
bathrooms                           float64
bed_type                             object
bedrooms                            float64
                                     ...   
amenities__Wheelchair_Accessible      int32
amenities__Family/Kid_Friendly        int32
amenities__Fire_Extinguisher          int32
amenities__Hangers                    int32
amenities__Smoking_Allowed            int32
Length: 67, dtype: object

In [148]:
data['bed_type'] = data['bed_type'].astype('category')
data['cancellation_policy'] = data['cancellation_policy'].astype('category')
data['instant_bookable'] = data['instant_bookable'].astype('category')
data['property_type'] = data['property_type'].astype('category')
data['room_type'] = data['room_type'].astype('category')
data['has_availability'] = data['has_availability'].astype('category')
data['host_is_superhost'] = data['host_is_superhost'].astype('category')
data['price'] = data['price'].astype('float')

In [149]:
df = pd.DataFrame(data.dtypes)
df.to_csv('cols.csv')

In [150]:
kernel = mf.ImputationKernel(
  data=data,
  save_all_iterations=True,
  random_state=10
)

kernel.mice(5,verbose=True)
#print(kernel)
completed_dataset = kernel.complete_data(dataset=0, inplace=False)

C:\Users\thari\AppData\Roaming\Python\Python39\site-packages\miceforest\ImputationKernel.py:369: UserWarning: [cancellation_policy,property_type] have very rare categories, it is a good idea to group these, or set the min_data_in_leaf parameter to preventlightgbm from outputting 0.0 probabilities.
  warn(


Initialized logger with name mice 1-5
Dataset 0
1  | bedrooms | host_is_superhost | host_listings_count | beds | bathrooms | review_scores_rating | review_scores_location | review_scores_checkin | review_scores_value | review_scores_communication
2  | bedrooms | host_is_superhost | host_listings_count | beds | bathrooms | review_scores_rating | review_scores_location | review_scores_checkin | review_scores_value | review_scores_communication
3  | bedrooms | host_is_superhost | host_listings_count | beds | bathrooms | review_scores_rating | review_scores_location | review_scores_checkin | review_scores_value | review_scores_communication
4  | bedrooms | host_is_superhost | host_listings_count | beds | bathrooms | review_scores_rating | review_scores_location | review_scores_checkin | review_scores_value | review_scores_communication
5  | bedrooms | host_is_superhost | host_listings_count | beds | bathrooms | review_scores_rating | review_scores_location | review_scores_checkin | review_

In [151]:
completed_dataset['bathrooms'].round(0)
completed_dataset['beds'].round(0)

0       3.0
1       1.0
2       1.0
3       1.0
4       3.0
       ... 
5829    2.0
5830    2.0
5831    2.0
5832    1.0
5833    1.0
Name: beds, Length: 5834, dtype: float64

In [152]:
completed_dataset['bathrooms'] = completed_dataset['bathrooms'].astype('int')
completed_dataset['beds'] = completed_dataset['beds'].astype('int')

In [154]:
completed_dataset.to_csv('preprocessed_dataset_1.csv')